# Text Summarization [T5 Fine Tune]

## 01. Install & Import Libraries

In [1]:
!pip install -U datasets transformers[torch] rouge-score evaluate accelerate nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-

In [ ]:
from huggingface_hub import notebook_login

# Paste hugging face token with write permission enabled and log in
notebook_login()

In [ ]:
# !apt install git-lfs

In [ ]:
import transformers
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

## 02. Finetuning a T5 model for summarization


### 2.1 Load Datasets

In [ ]:
import datasets
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("billsum")
metric = load("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
raw_datasets["test"][0]

{'text': "SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.\n\n    (a) Jackson County, Mississippi.--Section 219 of the Water \nResources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is \namended--\n        (1) in subsection (c), by striking paragraph (5) and inserting \n    the following:\n        ``(5) Jackson county, mississippi.--Provision of an alternative \n    water supply and a project for the elimination or control of \n    combined sewer overflows for Jackson County, Mississippi.''; and\n        (2) in subsection (e)(1), by striking ``$10,000,000'' and \n    inserting ``$20,000,000''.\n    (b) Manchester, New Hampshire.--Section 219(e)(3) of the Water \nResources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is \namended by striking ``$10,000,000'' and inserting ``$20,000,000''.\n    (c) Atlanta, Georgia.--Section 219(f)(1) of the Water Resources \nDevelopment Act of 1992 (106 Stat. 4835; 113 Stat. 335) is amended by \nstriking ``$25,000,000 for''.\n    (d) Pate

In [ ]:
import random
import pandas as pd

from IPython.display import display, HTML

# Function for random view of dataset elements
def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

In [ ]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

### 2.2 Data Preprocessing

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
print(tokenizer(text_target=["Hello, this one sentence!",
                             "This is another sentence being target."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 271, 2387, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
max_input_length = 1024
max_target_length = 64

prefix = "summarize: "

# Preprocess input data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

### 2.3 Fine-Tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Dynamic Padding batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=model_checkpoint,
                                       label_pad_token_id=tokenizer.pad_token_id)

# Load Pre-Trained T5 Checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]

# Define hyperparams
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-billsum",
    num_train_epochs=5,                       # No of epochs to train
    per_device_train_batch_size=batch_size,   # Batch size for training per device
    per_device_eval_batch_size=batch_size,    # Batch size for evaluation per device
    learning_rate=2e-5,                       # Learning rate for optimizer
    warmup_steps=500,                         # No of warmup steps for the learning rate scheduler
    weight_decay=0.01,                        # Weight decay coefficient for regularization
    logging_dir='./logs',                     # Directory for logging training information
    load_best_model_at_end=True,              # Whether to load the best model from checkpoints at the end of training
    logging_steps=500,                        # Log training metrics every `logging_steps` steps
    save_steps=1000,                          # Save model checkpoints every `save_steps` steps
    save_total_limit=2,                       # Save no of checkpoints
    eval_strategy="steps",                    # When to run evaluation during training: steps, epochs or none
    predict_with_generate=True,               # Generate preds during eval for metrics score
    generation_max_length=128,                # Generation length
    fp16=True,                                # Floating point 16 precision
    push_to_hub=True,                         # Save checkpoint in Hugging Face Hub
)

<!--  -->

In [ ]:
import nltk
import numpy as np
import evaluate

rouge = evaluate.load("rouge")
nltk.download('punkt')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # RougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print(predictions, labels)
    # print(tokenizer.pad_token_id)

    # Replace -100 in labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
from accelerate import Accelerator

# Initialize Accelerator and Trainer
Accelerator()
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.537300,1.730609,44.888400,29.904700,38.873200,39.709800,49.862649
1000,1.817500,1.516376,53.466300,36.893600,46.475600,47.613300,57.695320
1500,1.677500,1.440090,55.954900,38.796900,48.691800,49.921600,60.586724
2000,1.600000,1.401608,56.842300,39.597200,49.587700,50.808800,61.557969
2500,1.571700,1.373583,57.428200,40.212600,50.149800,51.381800,61.903334
3000,1.538900,1.357009,57.690900,40.504600,50.498700,51.776900,62.011624
3500,1.518300,1.342593,58.237200,41.147300,51.051700,52.342300,62.129703
4000,1.499000,1.331035,58.326000,41.256400,51.181700,52.451300,62.242276
4500,1.484500,1.323204,58.492500,41.542600,51.386500,52.694200,62.227593
5000,1.488800,1.320345,58.547500,41.586500,51.457400,52.791000,62.271031


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5925, training_loss=1.6424542210574895, metrics={'train_runtime': 8649.2623, 'train_samples_per_second': 10.954, 'train_steps_per_second': 0.685, 'total_flos': 2.564591796092928e+16, 'train_loss': 1.6424542210574895, 'epoch': 5.0})

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/luluw/t5-small-finetuned-billsum/commit/61a646b104e78adad1869d966257559a9c7a2f4a', commit_message='End of training', commit_description='', oid='61a646b104e78adad1869d966257559a9c7a2f4a', pr_url=None, pr_revision=None, pr_num=None)